In [22]:
import asyncio
import nest_asyncio
from decimal import Decimal
from typing import List
from starknet_py.hash.selector import get_selector_from_name
from starknet_py.net.client_models import Call
from starknet_py.net.full_node_client import FullNodeClient

class StarknetClient:
    """
    A client for making contract calls on the Starknet blockchain.
    """
    def __init__(self, node_url: str = "https://starknet-mainnet.public.blastapi.io"):
        self.node_url = node_url
        self.client = FullNodeClient(node_url=self.node_url)
    
    async def func_call(self, addr: int, selector: str, calldata: List[int], retries: int = 1) -> List[int]:
        """
        Make a call to a Starknet contract.
        
        Args:
            addr (int): Contract address.
            selector (str): Name of the function to call.
            calldata (List[int]): List of call arguments.
            retries (int, optional): Number of retry attempts.
        
        Returns:
            List[int]: The result of the contract call.
        """
        call = Call(
            to_addr=addr,
            selector=get_selector_from_name(selector),
            calldata=calldata,
        )
        for attempt in range(retries + 1):
            try:
                response = await self.client.call_contract(call, block_hash="latest")
                if hasattr(response, "result"):
                    return response.result
                elif isinstance(response, list):
                    return response
                else:
                    raise ValueError(f"Unexpected response type: {type(response)}")
            except Exception as e:
                if attempt == retries:
                    raise e
                await asyncio.sleep(10)


# Vesu protocol contract address 
VESU_CONTRACT = 0x02545b2e5d519fc230e9cd781046d3a64e092114f07e44771e0d719d148725ef
ETH_ADDRESS = 0x049d36570d4e46f48e99674bd3fcc84644ddd6b96f7c741b1562b82f9e004dc7
DAI_ADDRESS = 0x00da114221cb83fa859dbdb4c44beeaa0bb37c7537ad5ae66fe5e0efd20e6eb3


POOL_ID = 0x4dc4f0ca6ea4961e4c8373265bfd5317678f4fe374d76f3fd7135f57763bf28


get_pool_global_state: Retrieves whether deposits are enabled, and the total collateral and debt for an asset. The values are scaled using the asset_scale.


get_ltv_config: Retrieves the liquidation threshold for a collateral-debt pair (used to calculate health factor).

In [23]:
async def get_pool_global_state(client: StarknetClient, asset: int):
    """
    Retrieve global state for a given asset from the pool.
    Assumes that `asset_config` returns:
      [deposit_enabled, total_collateral, total_debt, asset_scale]
    
    Returns:
        deposit_enabled (int): 1 if deposits are enabled, 0 otherwise.
        total_collateral (Decimal): Total collateral (scaled to human-readable format).
        total_debt (Decimal): Total debt (scaled to human-readable format).
    """
    config = await client.func_call(VESU_CONTRACT, "asset_config", [POOL_ID, asset])
    deposit_enabled = config[0]
    total_collateral_raw = config[1]
    total_debt_raw = config[2]
    asset_scale = config[3]
    divisor = Decimal(10) ** Decimal(asset_scale)
    total_collateral = Decimal(total_collateral_raw) / divisor
    total_debt = Decimal(total_debt_raw) / divisor
    return deposit_enabled, total_collateral, total_debt

async def get_ltv_config(client: StarknetClient, collateral_asset: int, debt_asset: int):
    """
    Retrieve the LTV (Loan-To-Value) configuration for a collateral-debt pair.
    Assumes that `ltv_config` returns a list where the first element is the
    liquidation threshold in basis points.
    
    Returns:
        liquidation_threshold (Decimal): e.g., 0.75 means 75%.
    """
    ltv = await client.func_call(VESU_CONTRACT, "ltv_config", [POOL_ID, collateral_asset, debt_asset])
    liquidation_threshold = Decimal(ltv[0]) / Decimal(10000)
    return liquidation_threshold


The Health Factor Ratio indicates the safety of a borrowing position. A higher value means there is more collateral relative to debt, which implies a safer position.

Health Factor Ratio = (Total Collateral × Liquidation Threshold) / Total Debt

- Total Collateral: The total value of assets deposited as collateral.
- Liquidation Threshold:The percentage (as a decimal) of the collateral's value that can be used as security.
- Total Debt:The total outstanding borrowed amount.


A value above 1 indicates a safe position; below 1 implies risk of liquidation.


In [24]:
def calculate_health_factor(total_collateral: Decimal, total_debt: Decimal, liquidation_threshold: Decimal) -> Decimal:
    """
    Calculate the Health Factor Ratio:
    
    Health Factor = (Total Collateral × Liquidation Threshold) / Total Debt
    
    Returns a high value (e.g. 999) if total_debt is zero.
    """
    if total_debt == 0:
        return Decimal("999")
    adjusted_collateral = total_collateral * liquidation_threshold
    return adjusted_collateral / total_debt





The Interest Rate is derived from the rate accumulator. The rate accumulator increases over time to reflect accrued interest. The difference between the accumulator and 1 gives the accrued interest rate.

Interest Rate (%) = (Rate Accumulator - 1) × 100

- Rate Accumulator: A factor that typically starts at 1.0 and grows over time as interest accrues.
- Subtracting 1 gives the proportion of interest accrued, and multiplying by 100 converts it into a percentage.

In [25]:

def calculate_interest_rate(rate_accumulator: Decimal) -> Decimal:
    """
    Calculate an approximate interest rate from the rate accumulator.
    
    If the accumulator is 1.05, then the rate is roughly 5%.
    
    Returns:
        Interest rate as a percentage.
    """
    return (rate_accumulator - Decimal(1)) * 100

The Borrowing Potential is the extra amount that can be borrowed based on the available collateral. It is determined by comparing the adjusted collateral value (Total Collateral × Liquidation Threshold) to the current Total Debt.

Borrowing Potential = (Total Collateral × Liquidation Threshold) - Total Debt


- Total Collateral: The total deposited collateral value.
- Liquidation Threshold: The fraction of collateral value considered safe.
- Total Debt:The current borrowed amount.


A positive value indicates available borrowing capacity, whereas a zero (or negative) value means no additional borrowing is available.


In [26]:
def calculate_borrowing_potential(total_collateral: Decimal, total_debt: Decimal, liquidation_threshold: Decimal) -> Decimal:
    """
    Calculate the Borrowing Potential:
    
    Borrowing Potential = (Total Collateral × Liquidation Threshold) - Total Debt
    
    Returns zero if negative.
    """
    potential = (total_collateral * liquidation_threshold) - total_debt
    return potential if potential > 0 else Decimal(0)

This function gathers on-chain data for the given collateral and debt assets.
It calculates:
Health Factor Ratio: (Collateral × Liquidation Threshold) / Debt.
Interest Rate: Derived from the rate accumulator.
Deposit Status: Whether deposits are enabled for the collateral.
Borrowing Potential: Additional funds that can be borrowed.
Errors are caught and printed if the POOL_ID or data retrieval fails.

In [27]:
async def analyze_metrics_for_pair(collateral_asset: int, debt_asset: int):
    """
    Calculate and print protocol-level metrics for a collateral-debt pair.
    
    Metrics include:
      - Health Factor Ratio
      - Interest Rate (from the rate accumulator)
      - Deposit status (whether deposits are enabled)
      - Borrowing Potential
    
    This function retrieves on-chain data and performs the necessary calculations.
    """
    client = StarknetClient()
    
    try:
        deposit_enabled, total_collateral, _ = await get_pool_global_state(client, collateral_asset)
    except Exception as e:
        print("Error retrieving collateral asset state. Verify that POOL_ID is valid.")
        print("Error details:", e)
        return
    
    try:
        _, _, total_debt = await get_pool_global_state(client, debt_asset)
    except Exception as e:
        print("Error retrieving debt asset state. Verify that POOL_ID is valid.")
        print("Error details:", e)
        return

    try:
        
        liquidation_threshold = await get_ltv_config(client, collateral_asset, debt_asset)
    except Exception as e:
        print("Error retrieving LTV configuration. Verify that POOL_ID is valid.")
        print("Error details:", e)
        return
    
 
    health_factor = calculate_health_factor(total_collateral, total_debt, liquidation_threshold)
    
    
    try:
        rate_acc = await client.func_call(VESU_CONTRACT, "rate_accumulator", [POOL_ID, debt_asset])
        rate_acc_value = Decimal(rate_acc[0])
    except Exception as e:
        print("Error retrieving rate accumulator. Verify that POOL_ID is valid.")
        print("Error details:", e)
        return

    interest_rate = calculate_interest_rate(rate_acc_value)
    borrowing_potential = calculate_borrowing_potential(total_collateral, total_debt, liquidation_threshold)
    print("=== Metrics for Collateral (ETH) and Debt (DAI) ===")
    print(f"Deposit Enabled: {'Yes' if deposit_enabled == 1 else 'No'}")
    print(f"Total Collateral: {total_collateral:.4f}")
    print(f"Total Debt: {total_debt:.4f}")
    print(f"Liquidation Threshold: {liquidation_threshold:.4f}")
    print(f"Health Factor Ratio: {health_factor:.4f}")
    print(f"Interest Rate: {interest_rate:.2f}%")
    print(f"Borrowing Potential: {borrowing_potential:.4f}")


In [ ]:
async def main():
    await analyze_metrics_for_pair(ETH_ADDRESS, DAI_ADDRESS)

nest_asyncio.apply()
asyncio.run(main())
